# Example for:
Transfer Learning Empirical Experiment from all other classes of FashionMNIST to shoes (Sandal, Sneaker, Ankle boot) classes in FashionMNIST

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5,6]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64, # num channels for CNN
      'two_linear_layers': False,
      'hidden_dim_lin': 128,  # if two_linear_layers == True
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  # CHANGE: if no layer-wise lr
      'num_train': 40,
      'early_stop_patience': 6,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':4096,
      # DATASET PARAMS
      'pre_train_classes': [0, 1, 2, 3, 4, 6, 8],
      'fine_tune_classes': [5, 7, 9],
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      # default value for pooling_every_n_layers is 1 (after each cnn layer)
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': False,         # CHANGE: reinit the conv lyers only after the cut
      'reinit_both_dense': True   # CHANGE: True for reinitialize both dense layers, False for reinit only the last dense layer
    }

In [4]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.FashionMNIST, datasets.FashionMNIST, root_dir, transform=transform)## Pretraining

In [5]:
dataloader_wrapped.pretrain_train_loader.dataset.__len__()

37800

## Pretraining

In [6]:
#Create DNN model
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=7, bias=True)
)

In [7]:
# Train and evaluate - Skip if loading saved model!
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 35.97%
Validation Accuracy: 36.63%
Epoch: 1 	Training Accuracy: 59.17%
Validation Accuracy: 59.93%
Epoch: 2 	Training Accuracy: 69.57%
Validation Accuracy: 69.97%
Epoch: 3 	Training Accuracy: 76.40%
Validation Accuracy: 76.27%
Epoch: 4 	Training Accuracy: 79.24%
Validation Accuracy: 79.20%
Epoch: 5 	Training Accuracy: 79.97%
Validation Accuracy: 79.83%
Epoch: 6 	Training Accuracy: 78.35%
Validation Accuracy: 78.00%
Epoch: 7 	Training Accuracy: 82.17%
Validation Accuracy: 82.20%
Epoch: 8 	Training Accuracy: 83.48%
Validation Accuracy: 83.03%
Epoch: 9 	Training Accuracy: 83.65%
Validation Accuracy: 82.93%
Epoch: 10 	Training Accuracy: 84.15%
Validation Accuracy: 83.87%
Epoch: 11 	Training Accuracy: 84.84%
Validation Accuracy: 84.03%
Epoch: 12 	Training Accuracy: 85.16%
Validation Accuracy: 84.83%
Epoch: 13 	Training Accuracy: 85.78%
Validation Accuracy: 85.33%
Epoch: 14 	Training Accuracy: 86.27%
Validation Accuracy: 85.90%
Epoch: 15 	Training Accuracy: 86.53

In [8]:
torch.save(pretrained_model.state_dict(), 'pretrained_models/pretrained_cifar_arc/pretrained_model_toshoes.pth')

In [7]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_cifar_arc/pretrained_model_toshoes.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=7, bias=True)
)

In [8]:
eval(pretrained_model, device, dataloader_wrapped.val_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.3543, Accuracy: 3679.0/4200 (88%)

              precision    recall  f1-score   support

     Class 0       0.84      0.86      0.85       560
     Class 1       0.99      0.97      0.98       638
     Class 2       0.85      0.86      0.86       595
     Class 3       0.86      0.94      0.90       615
     Class 4       0.82      0.83      0.82       579
     Class 5       0.80      0.69      0.74       626
     Class 6       0.98      0.98      0.98       587

    accuracy                           0.88      4200
   macro avg       0.87      0.88      0.87      4200
weighted avg       0.88      0.88      0.87      4200



0.8759523809523809

In [16]:
# save model for later use
foldername = "pretrained_models/pretrained_normal_classes_new"
os.mkdir(foldername)
torch.save(pretrained_model.state_dict(), os.path.join(foldername, 'pretrained_model.pth'))

params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
#save params as well
with open(os.path.join(foldername, 'params.json'), 'w') as fp:
    json.dump(params_tmp, fp)

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [9]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]


In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 0.001, Lr: 0.001, Repeat: 0


KeyboardInterrupt: 

In [ ]:
print(baselines_results)

In [21]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results_jsons/baselines_freeze_{params["freeze"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [9]:
results = []
#percentages = [0.001, 0.01, 0.1, 0.5, 1.0]
percentages = [0.1]


In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Training Accuracy: 0.8864, Test Accuracy: 0.8854

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9049, Test Accuracy: 0.9070

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 2
Training Accuracy: 0.9407, Test Accuracy: 0.9338

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 3
Training Accuracy: 0.8741, Test Accuracy: 0.8772

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 4
Training Accuracy: 0.9068, Test Accuracy: 0.8946

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 5
Training Accuracy: 0.8821, Test Accuracy: 0.8821

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 6
Training Accuracy: 0.9099, Test Accuracy: 0.8971

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 7
Training Accuracy: 0.8957, Test Accuracy: 0.9038

Sampled Percentage: 0.1, Sampled Cut Point: 0, Lr: 0.00

In [11]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8864197530864197, 'test_acc': 0.8854444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9049382716049382, 'test_acc': 0.907}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9407407407407408, 'test_acc': 0.9337777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8740740740740741, 'test_acc': 0.8771666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9067901234567901, 'test_acc': 0.8946111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.8820987654320988, 'test_acc': 0.8821111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.9098765432098765, 'test_acc': 0.8971111111111111}, {'lr

In [12]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8864197530864197, 'test_acc': 0.8854444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9049382716049382, 'test_acc': 0.907}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9407407407407408, 'test_acc': 0.9337777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8740740740740741, 'test_acc': 0.8771666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9067901234567901, 'test_acc': 0.8946111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.8820987654320988, 'test_acc': 0.8821111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.9098765432098765, 'test_acc': 0.8971111111111111}, {'lr

In [27]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results_jsons/results_freeze_{params["freeze"]}_reinit_{params["reinit"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}.json', 'w') as f:
    json.dump(results, f)
results = results[1:]